# The detailed demonstration of the tropical rainfall diagnostic for low-resolution data:

# Calculation of histograms

### Content


1. [The load of packages, modules, and data](#1-load-of-packages-modules-and-data)

2. [Calculation of histogram of precipitation/tropical precipitation](#2-calculation-of-histogram-of-precipitationtropicalprecipitation)

    2.1. [with continuous  uniform binning](#21-with-continuous-uniform-binning)

    2.2  [with non-uniform (log-spaced) binning](#22-with-non-uniform-log-spaced-binning)

    2.3. [in the lazy (or delayed) mode](#23-in-the-lazy-or-delayed-mode)
    
    2.4. [with non-default tropical latitude band](#24-with-non-default-latitude-band)

    2.5  [with a specific time band](#25-with-a-specific-time-band)

    2.6. [with weights (weights=reader.grid_area)](#26-with-weights-weightsreadergrid_area)
 

3. [Loading the histogram to/from storage](#3-loading-the-histogram-tofrom-storage)

[Go to the end of file](#the-end)

#

## 1. The load of packages, modules, and data

In [1]:
from aqua import Reader
from tropical_rainfall import Tropical_Rainfall

Running tropical rainfall diagnostic...
Reading configuration yaml file..


#### ICON data

In [ ]:
reader = Reader(model="ICON", exp="ngc2009", source="lra-r100-monthly")
icon = reader.retrieve() 

## 2. Calculation of histogram of tropical precipitation


### 2.1. with continuous uniform binning 

##### We can perform calculations of the histograms with uniform binning by  initializing the following attributes of diagnostic: 
 - `num_of_bins`
 - `first_edge`
 - `width_of_bin`
 
 ##### In this case, all the bins in the histogram will have the width of bins.

In [5]:
diag = Tropical_Rainfall(num_of_bins = 20, first_edge = 0, width_of_bin = 1*10**(-6)/20)

##### We calculate the histogram simply by function:

In [8]:
hist_icon_trop  = diag.histogram_lowres(icon)
hist_icon_trop

/work/bb1153/b382267/mamba_dir/mambaforge/envs/aqua_common/lib/python3.11/site-packages/dask/array/core.py:3475: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(
2024-01-28 05:15:48 :: Trop. Rainfall :: WARNING  -> Amount of counts in the histogram is not equal to the size of the data
2024-01-28 05:15:49 :: Trop. Rainfall :: WARNING  -> The difference between the mean of the data and the mean of the histogram is                                     greater than the threshold. 
 Increase the number of bins and decrease the width                                         of the bins.


<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 2.5e-08 7.5e-08 ... 9.25e-07 9.75e-07
    width          (center_of_bin) float64 5e-08 5e-08 5e-08 ... 5e-08 5e-08
Data variables:
    counts         (center_of_bin) float64 5.797e+03 114.0 65.0 ... 6.0 11.0 7.0
    frequency      (center_of_bin) float64 0.01988 0.0003909 ... 2.401e-05
    pdf            (center_of_bin) float64 3.976e+05 7.819e+03 ... 754.5 480.1
    pdfP           (center_of_bin) float64 0.00994 0.0005864 ... 0.0004681
Attributes:
    history:            \n2024-01-28 05:15:39 AQUA💧: Retrieved from ICON_ngc2...
    time_band:          2020-01-20T00:00:00.000000000, 2022-03-01T00:00:00.00...
    lat_band:           -14.5, 14.5, freq=1.0
    lon_band:           0.0, 359.0, freq=1.0
    time_band_history:  2020-01-20T00:00:00.000000000, 2022-03-01T00:00:00.00...

##### The output of the histogram function is xarray.Dataset, which has two coordinates 
- `center_of_bin`:   the center of each bin
- `width`:           width of each bin
##### and contains three variables:
- `counts`:       the number of observations that fall into each bin
- `frequency`:    the number of cases in each bin, normalized by the total number of counts. The sum of the frequencies equals 1.
- `pdf`:          the number of cases in each bin, normalized by the total number of counts and width of each bin. 

##### local and global attributes. Local attributes contain the information about the time and space grid for which diagnostic performed the calculations:
- `time_band`:    the value of time of the first and last element in the dataset and the frequency of the time grid
- `lat_band`:     the maximum and minimum values of the tropical latitude band and the frequency of the latitude grid
- `lon_band`:     the maximum and minimum values of the longitude and the frequency of the longitude grid

##### Global attribute `history` contains the information about when the histogram was calculated and values of `time_band`, `lat_band`, and `lon_band`.

### 2.2.  with non-uniform (log-spaced) binning  

##### If you want to perform the calculation for non-uniform binning, use the `bins` attribute of the diagnostic instead of  `num_of_bins`, `first_edge`, and `width_of_bin`.  

In [5]:
bins = [1.00000000e-09, 1.63789371e-09, 2.68269580e-09, 4.39397056e-09,
       7.19685673e-09, 1.17876863e-08, 1.93069773e-08, 3.16227766e-08,
       5.17947468e-08, 8.48342898e-08, 1.38949549e-07, 2.27584593e-07,
       3.72759372e-07, 6.10540230e-07, 1.00000000e-06]

In [6]:
diag = Tropical_Rainfall(bins=bins)

In [7]:
hist_icon_trop_logspaced  = diag.histogram_lowres(icon)
hist_icon_trop_logspaced

/work/bb1153/b382289/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask/array/core.py:3469: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(


<xarray.Dataset>
Dimensions:        (center_of_bin: 14)
Coordinates:
  * center_of_bin  (center_of_bin) float64 1.319e-09 2.16e-09 ... 8.053e-07
    width          (center_of_bin) float64 6.379e-10 1.045e-09 ... 3.895e-07
Data variables:
    counts         (center_of_bin) float64 6.0 8.0 9.0 7.0 ... 117.0 195.0 251.0
    frequency      (center_of_bin) float64 0.008264 0.01102 ... 0.2686 0.3457
    pdf            (center_of_bin) float64 1.296e+07 1.055e+07 ... 8.877e+05

### 2.3. in the lazy (or delayed) mode

##### To perform calculations in the so-called lazy mode, use the flag `lazy` in the histogram function. 

In [8]:
diag = Tropical_Rainfall(num_of_bins = 20, first_edge = 0, width_of_bin = 1*10**(-6)/20)

In [9]:
hist_icon_lazy  = diag.histogram_lowres(icon, lazy=True) 
hist_icon_lazy

/work/bb1153/b382289/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask/array/core.py:3469: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(


<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 2.5e-08 7.5e-08 ... 9.25e-07 9.75e-07
    width          (center_of_bin) float64 5e-08 5e-08 5e-08 ... 5e-08 5e-08
Data variables:
    counts         (center_of_bin) float64 dask.array<chunksize=(20,), meta=np.ndarray>

##### `Note`: In the case of lazy calculation, the function's output will be different:  the xarray will contain only non-computed counts. If you want to add frequency and pdf variables to the histogram Dataset, apply the following function `add_frequency_and_pdf` (but only when you are actually ready to compute the histogram).

In [10]:
diag.add_frequency_and_pdf(tprate_dataset=hist_icon_lazy)

<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 2.5e-08 7.5e-08 ... 9.25e-07 9.75e-07
    width          (center_of_bin) float64 5e-08 5e-08 5e-08 ... 5e-08 5e-08
Data variables:
    counts         (center_of_bin) float64 dask.array<chunksize=(20,), meta=np.ndarray>
    frequency      (center_of_bin) float64 dask.array<chunksize=(20,), meta=np.ndarray>
    pdf            (center_of_bin) float64 dask.array<chunksize=(20,), meta=np.ndarray>

### 2.4. with non-default latitude band 

##### The tropical latitude band, by default, is the following

In [11]:
diag.trop_lat

10

##### You can modify the tropical latitude band as 

In [12]:
diag.trop_lat = 20
diag.trop_lat

20

##### You can also modify the latitude band with the use of an argument `trop_lat` of the histogram function 

In [13]:
hist_icon_trop  = diag.histogram_lowres(icon, trop_lat=30)
hist_icon_trop

/work/bb1153/b382289/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask/array/core.py:3469: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(


<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 2.5e-08 7.5e-08 ... 9.25e-07 9.75e-07
    width          (center_of_bin) float64 5e-08 5e-08 5e-08 ... 5e-08 5e-08
Data variables:
    counts         (center_of_bin) float64 1.622e+03 213.0 199.0 ... 81.0 80.0
    frequency      (center_of_bin) float64 0.4272 0.0561 ... 0.02133 0.02107
    pdf            (center_of_bin) float64 8.544e+06 1.122e+06 ... 4.214e+05

### 2.5. with a specific time band 

##### You can specify `s_time` and `f_time` as integers (starting and final times). For example, 

In [14]:
diag = Tropical_Rainfall(trop_lat = 30,  num_of_bins = 20, first_edge = 0, 
                         width_of_bin = 1*10**(-6)/15, s_time = 0, f_time = 47)

In [15]:
hist_icon = diag.histogram(icon)
hist_icon

<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 3.333e-08 1e-07 ... 1.233e-06 1.3e-06
    width          (center_of_bin) float64 6.667e-08 6.667e-08 ... 6.667e-08
Data variables:
    counts         (center_of_bin) float64 219.0 35.0 33.0 ... 34.0 33.0 32.0
    frequency      (center_of_bin) float64 0.2128 0.03401 ... 0.03207 0.0311
    pdf            (center_of_bin) float64 3.192e+06 5.102e+05 ... 4.665e+05

##### Also, you can specify `s_time` and `f_time` as strings. For example, 

In [16]:
diag.s_time = '2020:01'
diag.f_time ='2020/03/20/12'

##### or

In [17]:
diag.s_time = '2020'
diag.f_time ='2020.03.20'

##### There is the possibility of specifying only the year band or only the months' band. For example, we can select June, July, and August in a whole dataset as

In [18]:
diag.s_month = 3
diag.f_month = 6 

In order to evaluate the original dataset, we reset the time band at the full time range available in the dataset:

In [19]:
diag.s_time = None
diag.f_time = None
diag.s_month = None
diag.f_month = None

### 2.6. with weights (weights=reader.grid_area)

##### The function provides the opportunity to calculate the histogram with weights. Compared to standard methods, such computations are `high-speed` because they are based on `dask_histogram` package.

In [20]:
diag = Tropical_Rainfall(num_of_bins = 20, first_edge = 0, width_of_bin = 1*10**(-6)/15, 
                         s_time = 0, f_time = 47)

In [21]:
hist_icon_trop_weighted  = diag.histogram_lowres(icon, weights=reader.grid_area)
hist_icon_trop_weighted

/work/bb1153/b382289/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask/array/core.py:3469: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(
/work/bb1153/b382289/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask/array/core.py:3469: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(


<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 3.333e-08 1e-07 ... 1.233e-06 1.3e-06
    width          (center_of_bin) float64 6.667e-08 6.667e-08 ... 6.667e-08
Data variables:
    counts         (center_of_bin) float64 2.681e+12 4.297e+11 ... 3.937e+11
    frequency      (center_of_bin) float64 0.2119 0.03397 ... 0.03213 0.03112
    pdf            (center_of_bin) float64 3.179e+06 5.095e+05 ... 4.669e+05

##### You can also parse only the `bins` attribute of the class  instead of the specification of `num_of_bins`, `first_edge`, and `width_of_bin`. 

##### But such calculations would be much `slower`! (at around 50 times)

In [22]:
diag = Tropical_Rainfall(bins = bins)

In [23]:
hist_icon_trop_weighted  = diag.histogram_lowres(icon,  weights=reader.grid_area)
hist_icon_trop_weighted

/work/bb1153/b382289/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask/array/core.py:3469: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(
/work/bb1153/b382289/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask/array/core.py:3469: UserWarning: Passing an object to dask.array.from_array which is already a Dask collection. This can lead to unexpected behavior.
  warnings.warn(


<xarray.Dataset>
Dimensions:        (center_of_bin: 14)
Coordinates:
  * center_of_bin  (center_of_bin) float64 1.319e-09 2.16e-09 ... 8.053e-07
    width          (center_of_bin) float64 6.379e-10 1.045e-09 ... 3.895e-07
Data variables:
    counts         (center_of_bin) float64 7.324e+10 9.803e+10 ... 3.091e+12
    frequency      (center_of_bin) float64 0.008201 0.01098 ... 0.2688 0.3461
    pdf            (center_of_bin) float64 1.286e+07 1.051e+07 ... 8.887e+05

#

## 3. Loading the histogram to/from storage

#### Saving the histogram in storage. 

##### `Notice`: the name of the file will be unique and depends on the time band 

In [24]:
path_to_histogram='/work/bb1153/b382267/AQUA/histograms/' # please change this path to your own path
filename = diag.dataset_to_netcdf(dataset = hist_icon, path_to_netcdf = path_to_histogram, 
                                  name_of_file = 'icon')

2023-07-11 12:52:02 :: create_folder :: WARNING  -> Folder /work/bb1153/b382289/histograms/ already exists


#### Loading/opening the histogram from the storage 

##### The information about how to open and merge the list of histograms, see the notebook  `diagnostic_vs_streaming.ipynb`

In [25]:
diag.open_dataset(path_to_netcdf = filename)

<xarray.Dataset>
Dimensions:        (center_of_bin: 20)
Coordinates:
  * center_of_bin  (center_of_bin) float64 3.333e-08 1e-07 ... 1.233e-06 1.3e-06
    width          (center_of_bin) float64 ...
Data variables:
    counts         (center_of_bin) float64 ...
    frequency      (center_of_bin) float64 ...
    pdf            (center_of_bin) float64 ...

[Back to the top of file](#structure)

##### The end